In [1]:
import os
from typing import Literal

import geopandas as gpd
import pandas as pd

from datav import datav_geoatlas

df = pd.read_stata(os.path.expanduser("~/Documents/baseinfo_all_2020_所有注销的工业企业2003-2022(精简版).dta"))
df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df["经度_final"], df["纬度_final"], crs="EPSG:4326"))

In [ ]:
def get_adcode(df, level: Literal["province", "city", "district"]="province"):
    china = gpd.read_file(datav_geoatlas(adcode=100000, full=True)).dropna(subset="level").astype(
        {"adcode": int, "childrenNum": int, "subFeatureIndex": int}
    ).drop(columns=["adchar"])
    result = gpd.sjoin(df, china, how="left", predicate="within")
    try_nearest = result.query("name.isnull() and not 经度_final.isnull()")[df.columns]
    nearest = gpd.sjoin_nearest(try_nearest, china, how="left")
    result.loc[try_nearest.index, "adcode"] = nearest["adcode"]
    result.loc[try_nearest.index, "name"] = nearest["name"]
    if level == "province":
        return result[df.columns.tolist() + ["adcode", "name"]]
    results = []
    for pcode, group in result.groupby("adcode"):
        pcode = int(pcode)
        province = gpd.read_file(datav_geoatlas(adcode=pcode, full=pcode not in [710000]))
        province_name = group["name"].iloc[0]
        result = gpd.sjoin(group[df.columns], province, how="left", predicate="within")
        try_nearest = result.query("name.isnull() and not 经度_final.isnull()")[df.columns]
        nearest = gpd.sjoin_nearest(try_nearest, province, how="left", max_distance=0.1)
        result.loc[try_nearest.index, "adcode"] = nearest["adcode"]
        result.loc[try_nearest.index, "name"] = nearest["name"]
        result["province"] = province_name
        results.append(result)
    result = pd.concat(results)
    if level == "city":
        return result[df.columns.tolist() + ["adcode", "name", "province"]]
    districts = result[result["level"] == "district"]
    districts["city"] = ""
    cities = result[result["level"] == "city"]
    for ccode, group in cities.groupby("adcode"):
        city = gpd.read_file(datav_geoatlas(adcode=ccode, full=int(ccode) not in [419001]))
        city_name = group["name"].iloc[0]
        result = gpd.sjoin(group[df.columns], city)
        result["city"] = city_name
        districts = pd.concat([districts, result])
    return districts[df.columns.tolist() + ["adcode", "name", "city", "province"]]

p = get_adcode(df.query("not 经度_final.isnull()")).astype({"adcode": int})
p

Skipping field center: unsupported OGR type: 3
Skipping field centroid: unsupported OGR type: 3
Skipping field acroutes: unsupported OGR type: 1
/Users/tcztzy/GitHub/datav/.venv/lib/python3.9/site-packages/geopandas/array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


,RecordID,经度_final,纬度_final,geometry,adcode,name
0,39609536,112.893791,23.156624,POINT (112.89379 23.15662),440000,广东省
1,24044511,118.686211,41.001991,POINT (118.68621 41.00199),130000,河北省
2,38157212,121.343140,31.416922,POINT (121.34314 31.41692),310000,上海市
3,44227634,108.949188,19.233969,POINT (108.94919 19.23397),460000,海南省
4,10850108,116.125328,38.186329,POINT (116.12533 38.18633),130000,河北省
...,...,...,...,...,...,...
1562781,13178259,108.697861,30.928616,POINT (108.69786 30.92862),500000,重庆市
1562782,36912096,120.248520,30.182522,POINT (120.24852 30.18252),330000,浙江省
1562783,52187109,121.491493,30.921864,POINT (121.49149 30.92186),310000,上海市
1562784,5302301,119.688370,29.791410,POINT (119.68837 29.79141),330000,浙江省


In [40]:
df = pd.read_stata(os.path.expanduser("~/Documents/删除注销的工业企业的精简版2003-2022(编辑版).dta"))
df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df["经度_final"], df["纬度_final"], crs="EPSG:4326"))
p = get_adcode(df.query("not 经度_final.isnull()"), level="city").astype({"adcode": int})
p.drop(columns=["geometry"]).to_csv("删除注销的工业企业的精简版2003-2022(编辑版).csv", index=False)

Skipping field center: unsupported OGR type: 3
Skipping field centroid: unsupported OGR type: 3
Skipping field acroutes: unsupported OGR type: 1
/Users/tcztzy/GitHub/datav/.venv/lib/python3.9/site-packages/geopandas/array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(
Skipping field center: unsupported OGR type: 3
Skipping field centroid: unsupported OGR type: 3
Skipping field acroutes: unsupported OGR type: 1
/Users/tcztzy/GitHub/datav/.venv/lib/python3.9/site-packages/geopandas/array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(
Skipping field center: unsupported OGR type: 3
Skipping field centroid: unsupported OGR type: 3
Skipping field acroutes: unsup

In [42]:
gpd.read_file(datav_geoatlas(adcode=419001))

Skipping field center: unsupported OGR type: 3
Skipping field centroid: unsupported OGR type: 3
Skipping field acroutes: unsupported OGR type: 1


,adcode,name,childrenNum,level,parent,geometry
0,419001,济源市,0,city,"{ ""adcode"": 410000 }","MULTIPOLYGON (((112.05882 35.2799, 112.05869 3..."


In [22]:
df.query("not 经度_final.isnull()")

,RecordID,经度_final,纬度_final,geometry
0,39609536,112.893791,23.156624,POINT (112.89379 23.15662)
1,24044511,118.686211,41.001991,POINT (118.68621 41.00199)
2,38157212,121.343140,31.416922,POINT (121.34314 31.41692)
3,44227634,108.949188,19.233969,POINT (108.94919 19.23397)
4,10850108,116.125328,38.186329,POINT (116.12533 38.18633)
...,...,...,...,...
1562781,13178259,108.697861,30.928616,POINT (108.69786 30.92862)
1562782,36912096,120.248520,30.182522,POINT (120.24852 30.18252)
1562783,52187109,121.491493,30.921864,POINT (121.49149 30.92186)
1562784,5302301,119.688370,29.791410,POINT (119.68837 29.79141)


In [9]:
try_nearest = p.query("name.isnull() and not 经度_final.isnull()")
china = gpd.read_file(datav_geoatlas(adcode=100000, full=True)).dropna(subset="level").astype(
    {"adcode": int, "childrenNum": int, "subFeatureIndex": int}
).drop(columns=["adchar"])
gpd.sjoin_nearest(try_nearest, china, how="left")

Skipping field center: unsupported OGR type: 3
Skipping field centroid: unsupported OGR type: 3
Skipping field acroutes: unsupported OGR type: 1
/Users/tcztzy/GitHub/datav/.venv/lib/python3.9/site-packages/geopandas/array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


,RecordID,经度_final,纬度_final,geometry,adcode_left,name_left,index_right,adcode_right,name_right,childrenNum,level,parent,subFeatureIndex
3571,27597092,122.060242,29.747623,POINT (122.06024 29.74762),NaN,NaN,10,330000,浙江省,11,province,"{ ""adcode"": 100000 }",10
4195,23126201,122.082840,29.984076,POINT (122.08284 29.98408),NaN,NaN,10,330000,浙江省,11,province,"{ ""adcode"": 100000 }",10
16559,28988109,121.836220,29.992283,POINT (121.83622 29.99228),NaN,NaN,10,330000,浙江省,11,province,"{ ""adcode"": 100000 }",10
32895,9666100,121.897820,28.452637,POINT (121.89782 28.45264),NaN,NaN,10,330000,浙江省,11,province,"{ ""adcode"": 100000 }",10
40013,21182809,122.368698,30.473362,POINT (122.3687 30.47336),NaN,NaN,10,330000,浙江省,11,province,"{ ""adcode"": 100000 }",10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1431084,7963371,121.423309,32.527550,POINT (121.42331 32.52755),NaN,NaN,9,320000,江苏省,13,province,"{ ""adcode"": 100000 }",9
1462382,11869744,121.919487,28.457102,POINT (121.91949 28.4571),NaN,NaN,10,330000,浙江省,11,province,"{ ""adcode"": 100000 }",10
1478752,42116654,112.338692,16.831839,POINT (112.33869 16.83184),NaN,NaN,20,460000,海南省,19,province,"{ ""adcode"": 100000 }",20
1530819,27721105,122.070190,30.594328,POINT (122.07019 30.59433),NaN,NaN,10,330000,浙江省,11,province,"{ ""adcode"": 100000 }",10
